## 序文

今後、Scalaや設計について学ぶための一助とするために、 **デザインパターンを分解して、その上で再構築していこうと思います。**
このようなアプローチによりまとめていくように考えたのは、昨年からデザインパターンという概念について学ぶ際に

- クラス同士の関係が複雑に絡み合っていて理解し辛い
- 同じようなパターンも多くあるが、類似性、関連性が見え辛い
- 理解しても暗記のような覚え方になってしまい応用し辛い
- 応用できても適切な使用方法かわからず、柔軟に使う事が難しい
- Scalaだと(Singltonパターンなど)言語レベルにサポートしてるものもあるので、ScalaとGoFのデザインパターンを応用する必要がないところがある

などのデメリットを読んだ多くのの記事や本で感じたからです。

そこで今回は

- まず、Scalaは何ができるのかを（Scala学習者にはあたりまえなレベルまで）極力分解して列挙する
- 分解した構成要素を組み合わせる事でデザインパターンを構築していく

というアプローチによりまとめていきいたいと思います。
イメージとしては、数式1と数式2を組み合わせる事により、さらに数式3を構築でき、それが数式4に応用でき...
といった感じです。（故に、全ての構成要素にラベリングをしていきます。）

なので理想としては **「あたりまえじゃん」とおもえるレベルにからスタートして、その当たり前を組み合わせていく事で、最終的に「なるほどな」と読み手がおもえるようにしていきたいです**
このようなまとめ方によるメリットは

- スタートが初歩からはじまるので考えを整理しやすい
- 構成要素が分解されているおかげでTipsとして自身のレベルに応じて部分部分をつまみ食いしやすい
- 分解されているので、既存のパターンに捕われず、柔軟に応用できる

があります。（もちろんうまくまとめられれば、の話です。）
なお、ScalaDesignPatternという昨年発売した本がが参考にしたメインの本になる（予定）ですが、まだ未翻訳の本ですので、
明らかに間違った点がありましたら、本ではなく、長谷川の誤訳、誤読、理解不足などが原因です。ですので、突っ込んで頂けたらと思います。  
できるかな？やれるかな？

###  Unification
まず、大原則から。

- (1-1) Scalaでは全ての値は、オブジェクトである

- (1-2) 関数は第一級オブジェクトとして定義されている


したがって、(1-2)より、
- (1-3) 関数を変数として定義したり、メソッドやクラスの引数や戻り値として利用できる



In [39]:
// 例 1-1
val plus1=(x:Int)=> x+1

plus1(1)


plus1: Int => Int = <function1>
res38_1: Int = 2

なので、
- (1-4) オブジェクト指向言語としての特性と関数型言語としての特性の双方を利用できるため、多様な表現が可能になる。



##### Functions is classes
例1-1のようにScalaで関数がオブジェクトとして定義されているとは、コードを書いている時は感じないが、
裏では以下のようにclassが定義されており、applyされている。[Function1.scala](https://github.com/scala/scala/blob/v2.9.2/src/library/scala/Function1.scala#L1)

In [28]:
// 例1-2
val plus1= new Function1[Int,Int]{
    def apply(x:Int)=x+1
}

plus1(1)

plus1: Int => Int = <function1>
res27_1: Int = 2

つまり、裏側では関数はオブジェクトにインスタンス化されている。これにより (1-2)が実現されている。
この例により分かるように、

- (1-4) applyメソッドはScalaでは特別なメソッドであり，x.apply(y)はx(y)と同等である(糖衣構文)

なお、Function1は Traitである。Traitについては後述する



### Traits and Mixin Compositions
よりScalaらしいデザインパターンを構築するために、まず、*Trait*と*MixIn*の概念について説明する

#### Trait
Traitと言う概念が、出来る事、あるいは出来ない事を解説していく。
まず、

- () TraitはIntarfaceとして使用できる


In [29]:
// 例
trait Alarm{
    
    def trigger():String // 実装なし
}

defined trait Alarm

だが、JavaのInterfaceと大きく異なるのは次の点である

- () Traitは実装を持つ事が可能である


In [38]:
trait Notifier{
    val notificationMessage:String //実装なし
    
    // 実装あり
    def printNotification():Unit={
        println(notificationMessage)
    }
    
    def clear():Unit //実装なし
}

defined trait Notifier

- () Traitに実装を注入する事を、MixInという


In [ ]:
編集中

### Functional Design Pattern
() で説明したように、Scalaはオブジェクト指向と関数型のハイブリット言語である。なので、もちろん大抵のデザインパターンは応用できるが、 *関数型*
であるという側面を応用しなければ、完璧に使いこなせたとはいえない。
そこで、関数型言語の特性を
- () monoids
- () monads
- () functors

の三要素に分解し、そこから関数型言語を応用したデザインパターンの理解まで持っていく。



* 注意。以下の文章ではメソッドと関数を使い分けている事に注意。メソッドと関数は同じようだが、違う部分も多々ある。一つの大きな違いは、状態を
もつかもたないかである。メソッドの場合はオブジェクトとの結びつきが強く、Javaでいうstaticのように定義する事で状態をもつことが可能である。
状態を持つという事は、ときと場合により、同じメソッドを同じ引数を与えて呼び出しても同じ値を返すことはない。
それに対して関数は違う。状態をもたず、同じ引数を与えたら必ずおなじ値を返す。1+1は百回繰り返しても2である。


#### Monoids

() 何らかの計算処理により、値を加工する処理を行う事ができる概念を *演算子(operater)*と呼ぶ

演算子には例えば+ (1+1=2),- ×(1×2=2)など、さまざまな異なるタイプがあるように思うが、実はそれぞれの演算子は同じ法則に従うパターンに
まとめあげられる(抽象化)。まとめあげられるという事はプログラミング上、*再利用出来る可能性がでてくるということである。*
以下では、上記三要素に分類するために、それらの
要素をみたすルール(公理)をリストアップしていく

##### Monoidsとはなにか?
Monoidsとは、以下のルールに従う要素と演算子の集合(=型)である
- () Monoidに含まれる演算子は以下の法則に従う (結合則)  
    op(op(x,y),z)=op(x,op(y,z))  
    ex((1+(2+3))=((1+2)+3))
- () Monoidは以下の法則に従う要素(単位元)を持つ  
    op(x,zero)=op(zero,x)=x  
    ex(1+0=0+1=1)

まとめあげると、 
- () 結合則()に従う演算子と単位元()を要素に持つ型ならなんでもMonoidを名乗れる 


()より、例にある通り、(+,0,1,2,3)はMonoidとなる。当然、(+,0,1.1,-2)などもMonoidであり、2×1=1より、(×,1,2)もMonoid、さらにa+b=ab,a+""=a
とするならば(+,"",a,bもMonoidとなる。このように様々な演算子(+,×)と型(Int,Float,String)を組み合わせる事でMonoidがつくれる。
だが、(÷,1,2,3)はMonoidではない。なぜなら、3/(2/3)=3と(3/2)/3=9/2は違う。よって()をみたさないからである。
       
同じルールに従うという事は、*ポリモーフィックな関数やメソッドを作るという事が可能になる。* つまり、*Monoidを前提に作成した関数は
型が違っても全てのMonoidを引数として持つ事が可能である。*
       



()~()のルールを定義するtraitを作成する。
()のTraitの性質を利用して、上記のルールを抽象化し、Interfaceとして定義する。
こうする事で以下のMonoid[T]を継承したclassはすべてMonoidとなる

In [40]:
trait Monoid[T]{
    def op(l:T,r:T):T
    
    val zero:T
}

defined trait Monoid

このtraitを継承し、Monoidをつくる

In [17]:
object Monoids{
    val intAddition:Monoid[Int]=new Monoid[Int]{
        def op(l:Int,r:Int)=l+r
        val zero=0
    }
    
intAddition.op(1,2)
}

defined object Monoids

###### Monoidの使い方
Monoidがなんであるかはわかった。だが、重要なのは *Monoidがなにに使えるか* である。
編集中

In [ ]:
()　より、並列処理をしても結果が変わらない事が保証される

In [41]:
// (+,1,...1000)はMonad
(1 to 1000).par.reduce(_+_)

res40: Int = 500500

In [42]:
// (- ,1,...1000)はMonadではない
(1 to 1000).par.reduce(_- _) // shoule be -500498

res41: Int = -186000

In [45]:
したがって、Monad則は並列処理の結果を保証する

res44: Int = -499500

In [ ]:
### Traits and Mixin Compositions


In [ ]:
編集中

In [ ]:
### Abstract and Self Types

In [ ]:
### Polymorphism 
() ポリモーフィズムとは、同じメソッドを呼び出すが、オブジェクト毎にふるまいを変えることが出来る方法。
と定義する。

これを実現する手段はScalaには3種類ある

In [ ]:
#### SubType Polymorhism
() traitをインターフェースとして定義する事で実現する方法。

In [ ]:
#### Parametric Polymorphism
() 与えるパラメータ（Generics）によってふるまいを変える方法

In [10]:
class Container[T](data:T){
    def compare(other:T)=data.equals(other)
}

defined class Container

In [11]:
val numCon= new Container(1)
numCon.compare(1)

numCon: Container[Int] = cmd9$$user$Container@26a1e6c9
res10_1: Boolean = true

In [12]:
val strCon=new Container("a")
strCon.compare("b")

strCon: Container[String] = cmd9$$user$Container@1434ab15
res11_1: Boolean = false

In [ ]:
（）と（）より、
（）AbstractTypeとGenericsは同じ使い方ができる
（ただし、AbstractTypeはJavaには存在しない）

In [61]:
trait ContainerAT{
    type T
    val data:T
    def compare(other:T)=other.equals(data)
}

class StringContainer(val data:String) extends ContainerAT{
    type T=String
}

val strCon2= new StringContainer("a")

defined trait ContainerAT
defined class StringContainer
strCon2: $user.StringContainer = cmd60$$user$StringContainer@62c87bb4

In [ ]:
#### Ad hoc Polymorhism
() 暗黙の型変換を利用する方法


In [50]:
trait Adder[T]{
def sum(a:T,b:T):T
}


implicit object AdderClient{
    def sum[T:Adder](a:T,b:T):T=
    implicitly[Adder[T]].sum(a,b)
}

defined trait Adder
defined object AdderClient

In [61]:
// 上記のようにするだけだと、implicityでAdder[T]をAdder[Int]にすればよいのかAdder{String}にすればよいのか、
// わからないので、エラーが出る。よってAdderClientにぐたいてきな変換方法をおしえるひつようがある
AdderClient.sum(1,3)

: 

: 

In [62]:
trait Adder[T]{
def sum(a:T,b:T):T
}

object AdderClient2{

    def sum[T:Adder](a:T,b:T):T= implicitly[Adder[T]].sum(a,b)
   
}
// 上記のように、implicit変数を定義し、具体的な実装を与えてあげる事で、引数毎にsumのふるまいをかえることができる    
// implicityによる変換後の具体的なオブジェクトをimplicit val であたえてやる    
    implicit val int2Adder:Adder[Int]=new Adder[Int]{
        def sum(a:Int,b:Int):Int=a+b
    }
    
     implicit val str2Adder:Adder[String]=new Adder[String]{
        def sum(a:String,b:String):String=a+b
   
}


defined trait Adder
defined object AdderClient2
int2Adder: $user.Adder[Int] = cmd61$$user$$anonfun$1$$anon$1@1d1f8860
str2Adder: $user.Adder[String] = cmd61$$user$$anonfun$2$$anon$2@d5dbaf0

In [59]:
AdderClient2.sum(1,2)

res58: Int = 3

In [63]:
AdderClient2.sum("a","b")

res62: String = "ab"

In [ ]:
（）と（）により、
暗黙の変数と暗黙の型変換により、実装を注入することができる
よって、（）が実現可能になる

In [ ]:
### Self Type

() this:A=> と表記する事で、A内に定義されている定義ががそのクラス(トレイト)の中で使用可能になる

In [ ]:
次の例では永続化するための実装をもつDataBase永続化されるためのデータをラップするPersiterを定義する
Persisterは使用する側であり、DataBaseはPersisterに使用される側である。このように明確に責務が異なる場合、Self Typeは有用である

In [2]:
// DBのインターフェース
trait DataBase[T]{
    def save(data:T):Unit
}

// 実装
trait MemoryDataBase[T] extends DataBase[T]{
    val db:scala.collection.mutable.MutableList[T]=scala.collection.mutable.MutableList.empty
    
    def save(data:T)={
        println("Saved In Memory")
        db.+=:( data)
                     }
}

trait FileDataBase[T] extends DataBase[T]{
    
    def save(data:T)={
        println("Saved In File")
                     }
}

defined trait DataBase
defined trait MemoryDataBase
defined trait FileDataBase

In [5]:
// Self Typeでpersitを実装する

trait Persister[T]{
    this:DataBase[T]=>
    def persist(data:T):Unit=save(data)
}

defined trait Persister

In [ ]:
traitの()の性質により、Persiterをインスタンス化するときに実装を注入(DI)すればよい

In [7]:
val memoryPersister=new Persister[String] with MemoryDataBase[String]
val filePersister=new Persister[Int] with FileDataBase[Int]

memoryPersister.save("data")
filePersister.save(81) 

memoryPersister.db

Saved In Memory
Saved In File


memoryPersister: AnyRef with Persister[String] with MemoryDataBase[String] = cmd6$$user$$anonfun$3$$anon$1@7b4fa54e
filePersister: AnyRef with Persister[Int] with FileDataBase[Int] = cmd6$$user$$anonfun$4$$anon$2@67c48675
res6_4: collection.mutable.MutableList[String] = MutableList("data")

In [ ]:
Self Typeを使用せずに　、()のように、DataBaseを継承することも可能だが、そうすると

In [3]:
trait Persister[T] extends DataBase[T]{
    def persist(data:T):Unit=save(data)
}

defined trait Persiter

In [ ]:
となり、あたかもPersisterがDataBaseの一つの形であるように誤解されてしまい、
Persiter(使用する側)の責務とDataBase(使用される側)の責務がうまく分離できていない
Self Typeを使用する事で、DataBaseをPersiterに内包する事ができ、責務を分離できることから、Self Typeは有用である。
つまり、*SelfTypeはDI用の機能だといえる。*

In [ ]:
また、継承を使用すると、本来の責務上アクセスできないメソッドにアクセスできてしまうリスクがある。

In [12]:
// 以下よくない実装

//  DBに接続と切断をする
trait DB{
    def connect()={
        println("Connetct!")
    }
    
    def close()={
        println("close!")
    }
    
//     DBを強制終了する
        def drop()={
        println("drop!")
    }
}

//  UserDB
trait UserDB extends DB{
    
    def getUser(id:Int):Unit={
        connect()
        try{
            println(s"GetUser$id")
        }finally{
            close()
        }
    }
}

// UserDBからユーザー情報を取得するUserService

trait UserService extends UserDB{
    
    val user=getUser(1)
    
    // DB　　トレイトのメソッドをよべてしまう
    drop()
}

defined trait DB
defined trait UserDB
defined trait UserService

Connetct!
GetUser1
close!
drop!


badService: AnyRef with UserService = cmd16$$user$$anonfun$2$$anon$1@284b4e42

In [ ]:
上記のように継承に頼って、実装していくと、ServiceでDBトレイトのdropが呼び出し可能になってしまう=責務が漏れ出ている
ことにつながる。そこでSelfTypeを使用し

In [12]:
// SelfTypeによるリファクタリング

//  DBに接続と切断をする
trait DB2{
    def connect()={
        println("Connetct!")
    }
    
    def close()={
        println("close!")
    }
    
//     DBを強制終了する
        def drop()={
        println("drop!")
    }
}

//  UserDB
trait UserDB2{
    
    this:DB2=>
    def getUser(id:Int):Unit={
        connect()
        try{
            println(s"GetUser$id")
        }finally{
            close()
        }
    }
}

// UserDBからユーザー情報を取得するUserService

trait UserService2 {
    
    this:UserDB2 =>
   val user= getUser(1)

//     DB2トレイトのメソッドを呼ぼうとすると、コンパイルエラーになる
    drop()

    
}



: 

defined trait Monoid

: 

: 

: 

: 

In [ ]:
###### 